 **Fruit Quality Classification using CNN**

Loading ImageNet Base Model

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

Freeze Base Model

In [ ]:
base_model.trainable = False

Add Layers to Model

In [ ]:
inputs =  keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# pooling layer or flatten layer

x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


Compile Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Augment the Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect fruit to be upside-down so we will not flip vertically

Load Dataset

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('/content/drive/MyDrive/dataset/train',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('/content/drive/MyDrive/dataset/test',
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 10904 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.


Train the Model

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=5)

Epoch 1/5
340/340 [==============================] - 278s 777ms/step - loss: 0.9225 - accuracy: 0.7767 - val_loss: 0.2144 - val_accuracy: 0.9292
Epoch 2/5
340/340 [==============================] - 253s 741ms/step - loss: 0.1504 - accuracy: 0.9484 - val_loss: 0.1127 - val_accuracy: 0.9622
Epoch 3/5
340/340 [==============================] - 252s 739ms/step - loss: 0.1000 - accuracy: 0.9648 - val_loss: 0.0781 - val_accuracy: 0.9733
Epoch 4/5
340/340 [==============================] - 248s 729ms/step - loss: 0.0729 - accuracy: 0.9748 - val_loss: 0.0652 - val_accuracy: 0.9766
Epoch 5/5
340/340 [==============================] - 250s 734ms/step - loss: 0.0542 - accuracy: 0.9821 - val_loss: 0.0533 - val_accuracy: 0.9800


Unfreeze Model for Fine Tuning

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = keras.losses.CategoricalCrossentropy(from_logits=True) , metrics =[keras.metrics.CategoricalAccuracy()] )

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=5)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4847: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '


340/340 [==============================] - 343s 984ms/step - loss: 0.0520 - categorical_accuracy: 0.9839 - val_loss: 0.0261 - val_categorical_accuracy: 0.9907
Epoch 2/5
340/340 [==============================] - 330s 967ms/step - loss: 0.0206 - categorical_accuracy: 0.9941 - val_loss: 0.0104 - val_categorical_accuracy: 0.9967
Epoch 3/5
340/340 [==============================] - 328s 963ms/step - loss: 0.0088 - categorical_accuracy: 0.9963 - val_loss: 0.0037 - val_categorical_accuracy: 0.9981
Epoch 4/5
340/340 [==============================] - 328s 961ms/step - loss: 0.0051 - categorical_accuracy: 0.9986 - val_loss: 0.0095 - val_categorical_accuracy: 0.9963
Epoch 5/5
340/340 [==============================] - 330s 968ms/step - loss: 0.0061 - categorical_accuracy: 0.9986 - val_loss: 0.0059 - val_categorical_accuracy: 0.9981


Evaluate the Model

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

84/84 [==============================] - 49s 581ms/step - loss: 0.0080 - categorical_accuracy: 0.9989


[0.00797105673700571, 0.9988880753517151]